In [1]:
import pandas as pd

Load Labor, Race, and Income Demographics

In [4]:
labor = pd.read_csv("labor_demographics.csv")

In [6]:
race = pd.read_csv("ethnicity_income_demographics.csv")

Convert counts of ethnicity to rates.

In [27]:
race['Other'] = race['Other']/race['RACE-One-race']

In [28]:
race['Pacific Islander'] = race['Pacific Islander']/race['RACE-One-race']

In [29]:
race['Asian'] = race['Asian']/race['RACE-One-race']

In [30]:
race['Native American'] = race['Native American']/race['RACE-One-race']

In [31]:
race['African American'] = race['African American']/race['RACE-One-race']

In [35]:
race['White'] = race['White']/race['RACE-One-race']

Load Education Demographics

In [37]:
edu = pd.read_csv("education_demographics.csv")

2 Step Merge process to combine the 3 demographics files.

In [38]:
merge1 = labor.merge(race, left_on='Id2', right_on='Id2')

In [40]:
merge2 = merge1.merge(edu, left_on='Id2', right_on='Id2')

In [43]:
merge2.drop('Id_y', inplace = True, axis = 1)
merge2.drop('Geography_y', inplace = True, axis = 1)

Load Election Data by county from 2016.

In [20]:
election = pd.read_csv("election2016.csv")

In [82]:
election.tail()

,county,fips,cand,st,pct_report,votes,total_votes,lead,pct,state.name,Trump,Clinton,Other
18340,Niobrara County,56027,Hillary Clinton,WY,1.0,115,1297,Donald Trump,0.088666,Wyoming,0,115,0
18341,Niobrara County,56027,Gary Johnson,WY,1.0,49,1297,Donald Trump,0.037779,Wyoming,0,0,49
18342,Niobrara County,56027,Darrell Castle,WY,1.0,7,1297,Donald Trump,0.005397,Wyoming,0,0,7
18343,Niobrara County,56027,Jill Stein,WY,1.0,7,1297,Donald Trump,0.005397,Wyoming,0,0,7
18344,Niobrara County,56027,Rocky De La Fuente,WY,1.0,3,1297,Donald Trump,0.002313,Wyoming,0,0,3


Clean up by producing columns with vote totals for Trump, Clinton, and Others. Then group by county in order to have each row be a record of vote totals per county.

In [51]:
election['Trump'] = (election['cand'] == 'Donald Trump')*election['votes']

In [54]:
election['Clinton'] = (election['cand']=='Hillary Clinton')*election['votes']

In [53]:
election['Other'] = ((election['cand']!= 'Donald Trump')&(election['cand']!='Hillary Clinton'))*election['votes']

In [60]:
election_grp = election.groupby(['fips','county','st','state.name']).agg({'total_votes':'max', 
                                                           'Trump':'max', 'Clinton':'max', 'Other':'sum'}).reset_index()

In [85]:
election_grp.head(3)

,fips,county,st,state.name,Other,Clinton,Trump,total_votes
0,10001,Kent County,DE,Delaware,0.052752,0.44910,0.498148,74253
1,10003,New Castle County,DE,Delaware,0.049933,0.62304,0.327027,261468
2,10005,Sussex County,DE,Delaware,0.036649,0.37168,0.591670,105814


Create rates per candidate.

In [62]:
election_grp['Other'] = election_grp['Other']/election_grp['total_votes']

In [65]:
election_grp['Clinton'] = election_grp['Clinton']/election_grp['total_votes']

In [64]:
election_grp['Trump'] = election_grp['Trump']/election_grp['total_votes']

In [76]:
merge2['Id2'] = merge2['Id2'].apply(pd.to_numeric)

In [77]:
election_grp['fips'] = election_grp['fips'].apply(pd.to_numeric)

In [78]:
census_with_election = merge2.merge(election_grp, left_on = 'Id2', right_on = 'fips', how = 'outer')

In [80]:
census_with_election.to_csv('census_with_election_data.csv')